In [1]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt

In [2]:
from math import sin, cos, sqrt, atan2, radians

def distance_latlong(lat1,lon1,lat2,lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

    #print("Result:", distance)
    #print("Should be:", 278.546, "km")

In [3]:
apr_weather = pd.read_csv('APR_SG_hourly_weather.csv')
sg_geofabrik = pd.read_csv('sg_geofabrik.csv')
sg_geofabrik['geohash'] = sg_geofabrik['geohash'].str[:6]
train_agg = pd.read_csv('train_agg.csv')

In [4]:
sg_geofabrik.head()

,geohash,no_poi,maxspeed
0,w21xjf,1.0,70.0
1,w21xjz,1.0,NaN
2,w21xny,3.0,NaN
3,w21xpn,1.0,NaN
4,w21xpy,1.0,NaN


In [5]:
train_agg.head(10)

,trj_id,source_lat,source_lng,source_timestamp,dest_lat,dest_lng,dest_timestamp,time_taken,geohash_source,geohash_dest,hour,day
0,100,1.345079,103.938474,2019-04-16 02:44:15,1.335207,103.842207,2019-04-16 03:00:48,993.0,w21zthv,w21zdgp,2,1
1,1000,1.435317,103.788645,2019-04-11 00:49:33,1.368858,103.861007,2019-04-11 01:03:49,856.0,w23b1v2,w21zg3j,0,3
2,10004,1.433899,103.768705,2019-04-09 03:12:36,1.392301,103.908846,2019-04-09 03:26:47,851.0,w23b1m4,w21zut2,3,1
3,10005,1.371245,103.871188,2019-04-17 01:21:40,1.301454,103.843052,2019-04-17 01:38:48,1028.0,w21zg9s,w21z7hb,1,2
4,1001,1.344444,103.726811,2019-04-13 12:11:50,1.330114,103.875455,2019-04-13 12:30:11,1101.0,w21z8ke,w21zedp,12,5
5,10011,1.322871,103.697128,2019-04-20 08:06:26,1.375473,103.959461,2019-04-20 08:41:19,2093.0,w21xx8v,w21zvdk,8,5
6,10013,1.363236,103.842700,2019-04-08 21:22:48,1.348165,103.699219,2019-04-08 21:41:35,1127.0,w21zfbp,w21xxtr,21,0
7,1002,1.441166,103.801228,2019-04-21 23:11:38,1.280282,103.817199,2019-04-21 23:49:27,2269.0,w23b4n3,w21z63j,23,6
8,10027,1.338963,103.943973,2019-04-21 12:08:14,1.370300,103.845434,2019-04-21 12:33:13,1499.0,w21zt7c,w21zg13,12,6
9,1003,1.447680,103.812960,2019-04-19 01:30:44,1.333710,103.744187,2019-04-19 01:54:02,1398.0,w23b4rd,w21z8fb,1,4


In [6]:
train_agg['geohash_source'] = train_agg['geohash_source'].str[:6]
train_agg['geohash_dest'] = train_agg['geohash_dest'].str[:6]

In [12]:
weekend_busy = pd.read_csv('processed_csv/weekend_geohash_busy_6.csv')
weekend_busy = weekend_busy.groupby('node_trip').max().reset_index()
weekend_busy[['source', 'dest']] = weekend_busy.node_trip.str.split("-", expand=True)

weekend_normal = pd.read_csv('processed_csv/weekend_geohash_normal_6.csv')
weekend_normal = weekend_normal.groupby('node_trip').max().reset_index()
weekend_normal[['source', 'dest']] = weekend_normal.node_trip.str.split("-", expand=True)

weekday_busy = pd.read_csv('processed_csv/weekday_geohash_busy_6.csv')
weekday_busy = weekday_busy.groupby('node_trip').max().reset_index()
weekday_busy[['source', 'dest']] = weekday_busy.node_trip.str.split("-", expand=True)

weekday_normal = pd.read_csv('processed_csv/weekday_geohash_normal_6.csv')
weekday_normal = weekday_normal.groupby('node_trip').max().reset_index()
weekday_normal[['source', 'dest']] = weekday_normal.node_trip.str.split("-", expand=True)

main_df = pd.concat([weekend_busy, weekend_normal, weekday_busy, weekday_normal])
main_df = main_df.groupby('node_trip').max().reset_index()
main_df[['source', 'dest']] = main_df.node_trip.str.split("-", expand=True)

In [19]:
main_df.head()

,node_trip,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day,source,dest,geohash_source_2,no_poi_source,maxspeed_source,geohash_dest_2,no_poi_dest,maxspeed_dest,total_poi_geohash,maxspeed_mean_geohash
0,w21xjz-w21xjy,10452,w21xjz,w21xjy,2019-04-21 10:39:55,2019-04-21 10:40:38,43.0,10,6,w21xjz,w21xjy,w21xjz,1.0,70.0,w21xjy,0.0,70.0,1.0,70.0
1,w21xmb-w21xjz,10452,w21xmb,w21xjz,2019-04-21 10:38:41,2019-04-21 10:40:04,83.0,10,6,w21xmb,w21xjz,w21xmb,0.0,70.0,w21xjz,1.0,70.0,1.0,70.0
2,w21xmc-w21xmb,10452,w21xmc,w21xmb,2019-04-21 10:38:18,2019-04-21 10:38:45,27.0,10,6,w21xmc,w21xmb,w21xmc,0.0,50.0,w21xmb,0.0,70.0,0.0,60.0
3,w21xmf-w21xmg,44317,w21xmf,w21xmg,2019-04-08 09:13:18,2019-04-08 09:14:07,49.0,9,0,w21xmf,w21xmg,w21xmf,0.0,70.0,w21xmg,0.0,70.0,0.0,70.0
4,w21xmf-w21xq1,10452,w21xmf,w21xq1,2019-04-21 10:37:13,2019-04-21 10:38:06,53.0,10,6,w21xmf,w21xq1,w21xmf,0.0,70.0,w21xq1,0.0,70.0,0.0,70.0


In [14]:
main_df = pd.merge(main_df, sg_geofabrik, how='left', left_on='geohash_source', right_on='geohash')
main_df = pd.merge(main_df, sg_geofabrik, how='left', left_on='geohash_dest', right_on='geohash', suffixes=('_source', '_dest'))
main_df = main_df.fillna(0)
main_df['total_poi_geohash'] = main_df['no_poi_source'] + main_df['no_poi_dest']
main_df['maxspeed_mean_geohash'] = main_df[['maxspeed_source', 'maxspeed_dest']].mean(axis=1)

In [15]:
main_df.columns = ['node_trip', 'trj_id', 'geohash_source', 'geohash_dest',
       'timestamp_source', 'timestamp_dest', 'time_taken', 'hour', 'day',
       'source', 'dest', 'geohash_source_2', 'no_poi_source', 'maxspeed_source',
       'geohash_dest_2', 'no_poi_dest', 'maxspeed_dest', 'total_poi_geohash',
       'maxspeed_mean_geohash']

In [17]:
main_df = main_df.groupby('node_trip').max().reset_index()

In [18]:
len(main_df)

2961

In [20]:
train_poi_G = nx.Graph()
for line in main_df.iterrows():
    #print(line[1]['geohash_source'])
    #print(line[1]['geohash_dest'])
    #print(line[1]['total_poi_geohash'])
    train_poi_G.add_edge(line[1]['geohash_source'], line[1]['geohash_dest'], weight=line[1]['total_poi_geohash'])

In [21]:
train_speed_G = nx.Graph()
for line in main_df.iterrows():
    train_poi_G.add_edge(line[1]['geohash_source'], line[1]['geohash_dest'], weight=line[1]['maxspeed_mean_geohash'])

In [24]:
main_train_arr = []
for journey in train_agg.iterrows():
    
    temp_arr = []
    path = nx.dijkstra_path(train_poi_G,source=journey[1]['geohash_source'], target=journey[1]['geohash_dest'], weight='weight')
    total_poi_weight = 0
    for i in range(len(path) - 1):
        total_poi_weight += train_poi_G[path[i]][path[i+1]]['weight']

    path = nx.dijkstra_path(train_speed_G,source=journey[1]['geohash_source'], target=journey[1]['geohash_dest'], weight='weight')
    total_mean_speed = 0
    path_counter = 0
    for i in range(len(path) - 1):
        total_mean_speed += train_speed_G[path[i]][path[i+1]]['weight']
        path_counter += 1

    temp_arr.append(journey[1]['source_lat'])
    temp_arr.append(journey[1]['source_lng'])
    temp_arr.append(journey[1]['dest_lat'])
    temp_arr.append(journey[1]['dest_lng'])
    temp_arr.append(total_poi_weight)
    temp_arr.append(total_mean_speed/path_counter)
    temp_arr.append(journey[1]['hour'])
    temp_arr.append(journey[1]['day'])
    temp_arr.append(journey[1]['time_taken'])
    main_train_arr.append(temp_arr)

    


NodeNotFound: Source w21zth not in G

In [99]:
main_train_arr

[]

In [101]:
train_poi_G.nodes

NodeView(('w21xjz4', 'w21xjyf', 'w21xjz6', 'w21xjzg', 'w21xjze', 'w21xmbj', 'w21xmbh', 'w21xmbn', 'w21xmbq', 'w21xmbw', 'w21xmby', 'w21xmcr', 'w21xmcp', 'w21xmcx', 'w21xmfp', 'w21xmfr', 'w21xmfx', 'w21xq40', 'w21xq1b', 'w21xmfz', 'w21xmgp', 'w21xmgr', 'w21xmgx', 'w21xmgz', 'w21xmup', 'w21xmur', 'w21xqh2', 'w21xqh3', 'w21xqh8', 'w21xpqx', 'w21xpw8', 'w21xpqy', 'w21xpqz', 'w21xprm', 'w21xprj', 'w21xpw9', 'w21xpwm', 'w21xpwq', 'w21xpwr', 'w21xpy2', 'w21xpyd', 'w21xpyf', 'w21xpyg', 'w21xpz5', 'w21xpzh', 'w21xpzj', 'w21xpzm', 'w21xpzw', 'w21xpzy', 'w21xrbn', 'w21xq18', 'w21xqh6', 'w21xqhu', 'w21xqjh', 'w21xqj0', 'w21xqj1', 'w21xqj3', 'w21xqj9', 'w21xqjc', 'w21xqjf', 'w21xqn4', 'w21xqn1', 'w21xqn6', 'w21xqjj', 'w21xqjm', 'w21xqjt', 'w21xqjy', 'w21xqnn', 'w21xqmf', 'w21xqq4', 'w21xqn3', 'w21xqn9', 'w21xqn7', 'w21xqne', 'w21xqng', 'w21xqnu', 'w21xqp5', 'w21xqnf', 'w21xqp4', 'w21xqnv', 'w21xqph', 'w21xqnq', 'w21xqny', 'w21xqnw', 'w21xqnx', 'w21xqq8', 'w21xqnz', 'w21xqpp', 'w21xqp6', 'w21xqp7', 

In [102]:
train_agg.head()

,trj_id,source_lat,source_lng,source_timestamp,dest_lat,dest_lng,dest_timestamp,time_taken,geohash_source,geohash_dest,hour,day
0,100,1.345079,103.938474,2019-04-16 02:44:15,1.335207,103.842207,2019-04-16 03:00:48,993.0,w21zthv,w21zdgp,2,1
1,1000,1.435317,103.788645,2019-04-11 00:49:33,1.368858,103.861007,2019-04-11 01:03:49,856.0,w23b1v2,w21zg3j,0,3
2,10004,1.433899,103.768705,2019-04-09 03:12:36,1.392301,103.908846,2019-04-09 03:26:47,851.0,w23b1m4,w21zut2,3,1
3,10005,1.371245,103.871188,2019-04-17 01:21:40,1.301454,103.843052,2019-04-17 01:38:48,1028.0,w21zg9s,w21z7hb,1,2
4,1001,1.344444,103.726811,2019-04-13 12:11:50,1.330114,103.875455,2019-04-13 12:30:11,1101.0,w21z8ke,w21zedp,12,5


In [103]:
path = nx.dijkstra_path(train_poi_G,source='w21zthv', target='w21zdgp', weight='weight')

NetworkXNoPath: No path to w21zdgp.